In [ ]:
from florence_utils import run_inference
from llama_utils import call_llama_exp4,call_llama_combine_exp6

c:\Users\redti\anaconda3\envs\cvproject\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


c:\Users\redti\anaconda3\envs\cvproject\Lib\site-packages\fairscale\experimental\nn\offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
c:\Users\redti\anaconda3\envs\cvproject\Lib\site-packages\fairscale\experimental\nn\offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


[2025/01/19 20:51:52] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\redti/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\redti/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

In [ ]:
import os
import json
import shutil
from PIL import Image

# Directory containing the images
image_directory = "data/image"
output_data = []

# Create a temporary directory name
temp_dir = "temp_slices"

for i in range(1, 1005):
    try:
        img_path = f"{image_directory}/AMBER_{i}.jpg"

        # Open the image
        with Image.open(img_path) as img:
            width, height = img.size

            # Compute mid points (if not evenly divisible, the right/bottom gets the extra pixel)
            mid_w = width // 2
            mid_h = height // 2

            # Coordinates for cropping into 4 pieces
            # Top-left
            box1 = (0, 0, mid_w, mid_h)
            # Top-right
            box2 = (mid_w, 0, width, mid_h)
            # Bottom-left
            box3 = (0, mid_h, mid_w, height)
            # Bottom-right
            box4 = (mid_w, mid_h, width, height)

            # Ensure temp directory is clean
            if os.path.exists(temp_dir):
                shutil.rmtree(temp_dir)
            os.makedirs(temp_dir, exist_ok=True)

            # Crop and save the four pieces
            piece_paths = []
            for idx, box in enumerate([box1, box2, box3, box4], start=1):
                cropped_img = img.crop(box)
                cropped_path = os.path.join(temp_dir, f"slice_{idx}.jpg")
                cropped_img.save(cropped_path)
                piece_paths.append(cropped_path)

        # Generate captions for each piece
        caption_list = []
        for piece_path in piece_paths:
            object_detection_info = run_inference(piece_path, '<OD>')
            ocr_info = run_inference(piece_path, '<OCR>')
            caption = run_inference(piece_path, '<CAPTION>')
            piece_caption = call_llama_exp4(ocr_info,object_detection_info,caption)
            caption_list.append(piece_caption)

        # Cleanup temp directory
        shutil.rmtree(temp_dir)
        
       
        final_response = call_llama_combine_exp6(caption_list)
        
        print(f"Response for image {i}: {final_response}")
        output_data.append({"id": i, "response": final_response})

    except Exception as e:
        print(f"Issue on ID: {i}, Error: {e}")

# Save the results to a JSON file
output_file = "generated_captions/6_split_image.json"
with open(output_file, "w") as file:
    json.dump(output_data, file, indent=4)

print(f"Evaluation results saved to {output_file}")
